In [3]:
%load_ext autoreload

%autoreload 2

In [4]:
import numpy as np
import torch
import torchvision
import tqdm
from torch import nn
from torchvision import transforms as T
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import random

In [5]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [6]:
from torchvision.datasets import CIFAR10

In [7]:
train_transform = T.Compose([
    T.RandomCrop((32, 32), padding=4),
    T.RandomHorizontalFlip(0.5),
    T.ColorJitter(contrast=0.25),
    T.ToTensor(),
    T.Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.247, 0.243, 0.261)),
])
val_transform = T.Compose([
    T.ToTensor(),
    T.Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.247, 0.243, 0.261)),
])

train_dataset = CIFAR10("./data/", download=True, train=True, transform=train_transform)
val_dataset = CIFAR10("./data/", download=True, train=False, transform=val_transform)

Files already downloaded and verified
Files already downloaded and verified


In [8]:
train_dataloader = DataLoader(train_dataset, batch_size=100, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=100)

In [9]:
from src.utils import set_random_seed

In [10]:
from src.utils import train, predict

In [11]:
from src.layers.trl_masked import TRLMasked

set_random_seed(12345)
model = nn.Sequential(
    nn.Sequential(
        nn.Conv2d(3, 64, (3, 3), padding="same"),
        nn.BatchNorm2d(64),
        nn.ReLU(),
    ),
    nn.Sequential(
        nn.Conv2d(64, 64, (3, 3), padding="same"),
        nn.BatchNorm2d(64),
        nn.ReLU(),
    ),
    nn.MaxPool2d((3, 3), stride=(2, 2), padding=(1, 1)),
    nn.Sequential(
        nn.Conv2d(64, 128, (3, 3), padding="same"),
        nn.BatchNorm2d(128),
        nn.ReLU(),
    ),
    nn.Sequential(
        nn.Conv2d(128, 128, (3, 3), padding="same"),
        nn.BatchNorm2d(128),
        nn.ReLU(),
    ),
    nn.MaxPool2d((3, 3), stride=(2, 2), padding=(1, 1)),
    nn.Sequential(
        nn.Conv2d(128, 128, (3, 3), padding="same"),
        nn.BatchNorm2d(128),
        nn.ReLU(),
    ),
    nn.Sequential(
        nn.Conv2d(128, 128, (3, 3), padding="same"),
        nn.BatchNorm2d(128),
        nn.ReLU(),
    ),
    nn.Sequential(
        TRLMasked((128, 8, 8), (1, 1, 1), 10),
    )
)

optimizer = torch.optim.SGD(model.parameters(), lr=1e-2, momentum=0.9)
criterion = nn.CrossEntropyLoss(reduction="mean")
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 30, gamma=0.1, verbose=True)
# scheduler = None
n_epochs = 100

device = "cuda:0" if torch.cuda.is_available() else torch.device("cpu")
# device = "cpu"
model = model.to(device)

Adjusting learning rate of group 0 to 1.0000e-02.


In [17]:
# model = torch.load("./models/test_model.pt")
# model[-1][0].clear_masks()

In [18]:
import tensornetwork as tn
import numpy as np
from copy import deepcopy

def append_to_axis(data, axis, tensor=None, device="cuda:0"):
    with torch.no_grad():
        std = torch.sqrt(torch.norm(data)**2 / np.prod(data.shape))
        if tensor is not None:
            return torch.cat((data, tensor), dim=axis)
        zeros_shape = list(data.shape)
        zeros_shape[axis] = 1
        return torch.cat((data, torch.normal(0, std, zeros_shape, device=device)), dim=axis)

def decrease_node(tensor, axis):
    with torch.no_grad():
        indices = [slice(None) for _ in tensor.data.shape]
        indices[axis] = slice(-1, None)
        tensor_slice = tensor.data[indices].detach().clone()
        indices[axis] = slice(-1)
        tensor.data = tensor.data[indices].clone()
        return tensor_slice

def decrease_edge(edge: tn.Edge, device="cuda:0"):
    return (decrease_node(edge.node1.tensor, edge.axis1), decrease_node(edge.node2.tensor, edge.axis2))

def increase_edge(edge: tn.Edge, tensors=(None, None), device="cuda:0"):
    with torch.no_grad():
        edge.node1.tensor.data = append_to_axis(edge.node1.tensor.data, edge.axis1, tensors[0], device=device)
        mask1 = torch.zeros_like(edge.node1.tensor.data, device="cpu", dtype=torch.long)
        mask1.index_fill_(edge.axis1, torch.LongTensor([-1]), 1)
        edge.node2.tensor.data = append_to_axis(edge.node2.tensor.data, edge.axis2, tensors[1], device=device)
        mask2 = torch.zeros_like(edge.node2.tensor.data, device="cpu", dtype=torch.long)
        mask2.index_fill_(edge.axis2, torch.LongTensor([-1]), 1)
        return mask1.to(device), mask2.to(device)

def increase_edge_in_layer(layer, edge, device="cuda:0"):
    with torch.no_grad():
        cloned_layer = deepcopy(layer)
        cloned_edge = [e for e in tn.get_all_nondangling(cloned_layer.construct_network()) 
            if e.node1.name == edge.node1.name and e.node2.name == edge.node2.name][0]
        cloned_layer.masks = {}
        mask1, mask2 =increase_edge(cloned_edge)
        cloned_layer.masks[edge.node1.name] = mask1
        cloned_layer.masks[edge.node2.name] = mask2

        return cloned_layer, cloned_edge


In [19]:
def choose_and_increase_edge(model, tensor_network, train_edge):
    edges = list(tn.get_all_nondangling(tensor_network))
    edges_results = []
    old_layer = model[-1][0]
    for edge in edges:
        model[-1][0], cloned_edge = increase_edge_in_layer(old_layer, edge)

        for param in model.parameters():
            param.requires_grad = False
        cloned_edge.node1.tensor.requires_grad = True
        cloned_edge.node2.tensor.requires_grad = True

        print("Train edge {}".format(edge.name))
        final_loss = train_edge(model)
        model[-1][0].clear_masks()
        edges_results.append((model[-1][0], final_loss, edge.name))
    model[-1][0], _, edgename = min(edges_results, key=lambda x: x[1])
    print("Choosen edge {}".format(edgename))

In [20]:
def train_edge(model):
    optimizer = torch.optim.SGD(model.parameters(), lr=1e-2, momentum=0.9)
    train(model, train_dataloader, val_dataloader, criterion, optimizer, scheduler=None, n_epochs=2, device=device)
    all_losses, _, _ = predict(model, train_dataloader, criterion, device)
    return np.sum(all_losses)

for i in range(30):
    for param in model.parameters():
        param.requires_grad = True
    optimizer = torch.optim.SGD(model.parameters(), lr=1e-2, momentum=0.9)
    print("Train full model")
    train(model, train_dataloader, val_dataloader, criterion, optimizer, scheduler=None, n_epochs=1, device=device)
    choose_and_increase_edge(model, model[-1][0].construct_network(), train_edge)

Train full model


  0%|          | 0/500 [00:00<?, ?it/s]

Epoch: 0, Train loss: 1.9307632489204407, Validation loss: 1.789718461036682, Validation accuracy: 0.2599
Train edge rank_1


  0%|          | 0/500 [00:00<?, ?it/s]

Epoch: 0, Train loss: 1.78416850566864, Validation loss: 1.7244648015499116, Validation accuracy: 0.2853


...

Epoch: 1, Train loss: 0.13076744908466936, Validation loss: 0.43075027465820315, Validation accuracy: 0.8878
Train edge rank_0


  0%|          | 0/500 [00:00<?, ?it/s]

Epoch: 0, Train loss: 0.1360051382854581, Validation loss: 0.4308974747359753, Validation accuracy: 0.8884


  0%|          | 0/500 [00:00<?, ?it/s]

Epoch: 1, Train loss: 0.12968290810286998, Validation loss: 0.4222817727923393, Validation accuracy: 0.8907
Choosen edge rank_2


In [24]:
torch.save(model, "./models/greedy-tn-intermediate.pt")

In [12]:
model = torch.load("./models/greedy-tn-intermediate.pt")

In [13]:
for core in model[-1][0].construct_network():
    print(core.shape)

(128, 25)
(8, 5)
(8, 3)
(25, 5, 3, 10)


In [27]:
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=0.9)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 30, gamma=0.1, verbose=True)

for param in model.parameters():
    param.requires_grad = True

accuracies = train(model, train_dataloader, val_dataloader, criterion, optimizer, device, 70, scheduler, plot=False)

Adjusting learning rate of group 0 to 1.0000e-03.


  0%|          | 0/500 [00:00<?, ?it/s]

Adjusting learning rate of group 0 to 1.0000e-03.
Epoch: 0, Train loss: 0.10069047795608639, Validation loss: 0.42230992525815964, Validation accuracy: 0.9006


  0%|          | 0/500 [00:00<?, ?it/s]

Adjusting learning rate of group 0 to 1.0000e-03.
Epoch: 1, Train loss: 0.08073057451471687, Validation loss: 0.46170606195926667, Validation accuracy: 0.9023


...

Adjusting learning rate of group 0 to 1.0000e-05.
Epoch: 67, Train loss: 0.021761620508390478, Validation loss: 0.7134135715663433, Validation accuracy: 0.9069


  0%|          | 0/500 [00:00<?, ?it/s]

Adjusting learning rate of group 0 to 1.0000e-05.
Epoch: 68, Train loss: 0.022366122332401574, Validation loss: 0.712723551467061, Validation accuracy: 0.9066


  0%|          | 0/500 [00:00<?, ?it/s]

Adjusting learning rate of group 0 to 1.0000e-05.
Epoch: 69, Train loss: 0.02334692127339076, Validation loss: 0.7121564545482397, Validation accuracy: 0.9071


In [ ]:
all_losses, predicted_labels, true_labels = predict(model, val_dataloader, criterion, device)
assert len(predicted_labels) == len(val_dataset)
accuracy = accuracy_score(predicted_labels.to("cpu"), true_labels.to("cpu"))
print("tests passed")

In [ ]:
torch.save({
    "model": model.state_dict(),
}, "./models/conv-trl-2.1-state-dict.pt")